In [31]:
import pandas as pd

data = [[1, 1, '2020-06-01', 1, 10], 
        [2, 1, '2020-06-08', 2, 10], 
        [3, 2, '2020-06-02', 1, 5], 
        [4, 3, '2020-06-03', 3, 5], 
        [5, 4, '2020-06-04', 4, 1], 
        [6, 4, '2020-06-05', 5, 5], 
        [7, 5, '2020-06-05', 1, 10], 
        [8, 5, '2020-06-14', 4, 5], 
        [9, 5, '2020-06-21', 3, 5]]
orders = pd.DataFrame(data, 
                      columns=['order_id', 
                               'customer_id', 
                               'order_date', 
                               'item_id', 
                               'quantity']).astype({'order_id':'Int64', 
                                                    'customer_id':'Int64', 
                                                    'order_date':'datetime64[ns]', 
                                                    'item_id':'object', 'quantity':'Int64'})

data = [[1, 'LC Alg. Book', 'Book'], 
        [2, 'LC DB. Book', 'Book'], 
        [3, 'LC SmarthPhone', 'Phone'], 
        [4, 'LC Phone 2020', 'Phone'], 
        [5, 'LC SmartGlass', 'Glasses'], 
        [6, 'LC T-Shirt XL', 'T-shirt']]
items = pd.DataFrame(data, columns=['item_id', 
                                    'item_name', 
                                    'item_category']).astype({'item_id':'object', 
                                                              'item_name':'object', 
                                                              'item_category':'object'})
display(orders, items)

,order_id,customer_id,order_date,item_id,quantity
0,1,1,2020-06-01,1,10
1,2,1,2020-06-08,2,10
2,3,2,2020-06-02,1,5
3,4,3,2020-06-03,3,5
4,5,4,2020-06-04,4,1
5,6,4,2020-06-05,5,5
6,7,5,2020-06-05,1,10
7,8,5,2020-06-14,4,5
8,9,5,2020-06-21,3,5


,item_id,item_name,item_category
0,1,LC Alg. Book,Book
1,2,LC DB. Book,Book
2,3,LC SmarthPhone,Phone
3,4,LC Phone 2020,Phone
4,5,LC SmartGlass,Glasses
5,6,LC T-Shirt XL,T-shirt


In [32]:
orders = orders.merge(items, how="left")
orders["day"] = orders["order_date"].dt.day_name()

unique_categories = sorted(items["item_category"].unique().tolist())
orders["item_category"] = orders["item_category"].astype("category").cat.set_categories(unique_categories)
orders["day"] = orders["day"].astype("category").cat.set_categories(['Monday', 'Tuesday', 'Wednesday', 
                                                                     'Thursday', 'Friday', 'Saturday', 'Sunday'])
orders = pd.pivot_table(orders, 
                        index="item_category",
                        columns="day",
                        values="quantity",
                        aggfunc="sum",
                        observed=False)
orders = orders.reset_index(names="category").sort_values(by=["category"])

orders

day,category,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Book,20,5,0,0,10,0,0
1,Glasses,0,0,0,0,5,0,0
2,Phone,0,0,5,1,0,0,10
3,T-shirt,0,0,0,0,0,0,0
